In [ ]:
import os
import sys
sys.path.insert(0, "..")
import gc
gc.enable()


In [ ]:

import os
import sys
sys.path.insert(0, os.getcwd())
from utils.DataLoader import  DataLoader
from utils.utilities import *
from utils.visualization import *
from models.pspnet import PSPNet
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.debugging.set_log_device_placement(True)
dl = DataLoader()

In [ ]:
train_df, valid_df = dl.train_split(0.2, filterNan=True)
print("train set shape", train_df.shape)
print("validation set shape", valid_df.shape)

In [ ]:
grouped_images_gen = dl.get_grouped_images_gen(train_df)#, f="adaptive_equalization") # applying filters increases dramatically the execution time
aug_gen = get_augmented_images_generator(grouped_images_gen)
valid_x, valid_y = next(dl.get_grouped_images_gen(valid_df, batch_size=500))#, f="adaptive_equalization")) #valid_df.shape[0]))

In [ ]:
t_x, t_y = next(aug_gen)
x_shape =np.squeeze(t_x[0]).shape
psp_model = PSPNet(x_shape)
psp_model.compile()

In [ ]:
hl = psp_model.validate(aug_gen, train_df.shape[0], valid_set=(valid_x, valid_y), epochs=3)
psp_model.show_loss(hl)

In [ ]:
fig, m_axs = plt.subplots(50, 3, figsize = (10, 100))
for i, (ax1, ax2, ax3) in enumerate(m_axs):
    test_img = np.expand_dims(valid_x[i], 0)
    y = psp_model.infer(test_img)
    ax1.imshow(valid_x[i])
    ax1.set_title('Image')
    ax2.imshow(valid_y[i])
    ax2.set_title('Mask')
    ax3.imshow(y[0, :, :, 0], vmin = 0, vmax = 1)
    ax3.set_title('Prediction')
plt.show()